# iSCAT Analysis Demo

This notebook demonstrates:
* Loading mulitple tracks from a single file
* Retreiving single tracks from this
* ADC analysis of a single track
* ADC analysis of multiple tracks at once
* Retreiving analysis results using `adc_summary`
* Retreiving tracks that fit a certain diffusion category

Before we start fitting our data, we need to add some models. `iscat_lib.analysis.models` contains a few models that we can add to `ModelDB`. All models added this way will be used during analysis.

In [1]:
from iscat_lib.analysis import ModelDB
from iscat_lib.analysis.models import ModelBrownian, ModelConfined, ModelHop

ModelDB().add_model(ModelBrownian)
ModelDB().add_model(ModelConfined)
ModelDB().add_model(ModelHop)

Multiple tracks are stored in a `ListOfTracks` objects. 

In [2]:
from iscat_lib.analysis import ListOfTracks

Load the tracks from some example data. We can use `tracks.normalize` to set the initial time value of all tracks to 0.

In [3]:
tracks = ListOfTracks.from_file("../examples/more test data/180912 20nm GoldNP on 75DOPC25Chol (2kHz 100pc power)_Position_converted_fixed.csv", unit_length='micrometres')
tracks.normalize(normalize_t = True, normalize_xy = False)

Single tracks are stored in a `Track` object.

In [4]:
from iscat_lib.analysis import Track

We can retreive single tracks either by their index in then `LiftOfTracks` object using `get_track` of by their ID (should they have one) using `get_track_by_id`:

In [5]:
single_track = tracks.get_track(0)

We can now do ADC analysis on the track:

In [6]:
results = single_track.adc_analysis()

Analysis results like the calculated values for $D_{app}$, fit parameters and much more are returned in a dictionary. We can also retreive the dictionary of the last analysis at any time with `get_adc_analysis_results`.

In [7]:
fit_results = results["fit_results"]
fit_results

{'ModelBrownian': {'params': array([8.72791175e-13, 5.10762742e-08]),
  'errors': array([4.19813777e-15, 8.78994339e-10]),
  'bic': -54.64141697148811,
  'rel_likelihood': 1.0},
 'ModelConfined': {'params': array([1.04841983e-21, 1.04105487e-07, 4.26091324e-04]),
  'errors': array([1.22687120e-11, 1.20631910e-08, 5.83251588e-23]),
  'bic': -49.550082766762905,
  'rel_likelihood': 0.07842071987971853},
 'ModelHop': {'params': array([8.88467225e-13, 8.81264679e-12, 3.69141356e-08, 3.35368576e-04]),
  'errors': array([7.07694003e-15, 2.56410219e-11, 4.99850559e-09, 5.95617034e-04]),
  'bic': -53.553995514604864,
  'rel_likelihood': 0.5805898389792749}}

In order to set initial parameters or bounds for the fits, we need to modify the models inside `ModelDB`. These will then be applied during all analysis from this point on.

In [8]:
ModelDB().get_model(ModelBrownian).initial = fit_results["ModelBrownian"]["params"]
ModelDB().get_model(ModelConfined).initial = fit_results["ModelConfined"]["params"]
ModelDB().get_model(ModelHop).initial = fit_results["ModelHop"]["params"]

Here, we set all initial parameters to the results of our single fit.

In [9]:
ModelDB().get_model(ModelBrownian).initial

array([8.72791175e-13, 5.10762742e-08])

Now that we set our initial guesses, let's analyse the remaining tracks at once.

Enabling logarithmic sampling is a good idea since the time axis will be scaled logarithmically by default. We can also set the maximum time on the time for which to fit.

`adc_analysis` will return a list containing the indices of all tracks for which a fit has failed. These can then be retreived with `get_track` and analysed further.

In [ ]:
tracks.adc_analysis(fit_max_time=50e-3, enable_log_sampling=True)

`adc_summary` gives an overview of the analysis results including optional plots, the averaged parameters for each model, the averaged MSD for each model and the averaged $D_{app}$ for each model.

In [ ]:
tracks.adc_summary(plot_dapp = True, plot_pie_chart=True)

Now that analysis is done we can also retreive all tracks that fit a certain diffusion category best:

In [ ]:
tracks_brownian = tracks.get_sublist(method="adc", model=ModelBrownian)
tracks_brownian.adc_summary(plot_dapp=True)

As mentioned before, we can retreive the analysis results for any track, at any time:

In [ ]:
tracks_brownian.get_track(42).get_adc_analysis_results()

We can also plot them:

In [ ]:
tracks_brownian.get_track(42).plot_adc_analysis_results()